
# Stage 04 — Data Acquisition & Ingestion

This notebook pulls **one market-related dataset via API** (FRED) and **scrapes one public table** (Wikipedia), validates each, and saves timestamped CSVs to `data/raw/`.



## 0) Setup
**Repo structure expected:**
```
data/raw/
notebooks/stage04_data-acquisition-and-ingestion.ipynb
.env           # (not committed)
.env.example
.gitignore
```
Ensure `.env` contains `FRED_API_KEY=...`.


In [ ]:

from __future__ import annotations
import os
from pathlib import Path
from datetime import datetime, timezone
from typing import Iterable

import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Paths
PROJECT_ROOT = Path.cwd().resolve().parent if (Path.cwd().name == "notebooks") else Path.cwd()
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_RAW.mkdir(parents=True, exist_ok=True)

# Load secrets
load_dotenv()
print("Project root:", PROJECT_ROOT)
print("Raw data dir:", DATA_RAW)


In [ ]:

def ts_stamp() -> str:
    return datetime.now(timezone.utc).strftime("%Y%m%d-%H%M")

def save_csv(df: pd.DataFrame, prefix: str, *parts: str) -> Path:
    name = "_".join([prefix, *[p.replace(" ", "-") for p in parts], ts_stamp()]) + ".csv"
    out = DATA_RAW / name
    df.to_csv(out, index=False)
    print("Saved:", out)
    return out

def expect_columns(df: pd.DataFrame, required: Iterable[str]) -> None:
    missing = [c for c in required if c not in df.columns]
    assert not missing, f"Missing required columns: {missing}"

def assert_no_all_na(df: pd.DataFrame, cols: Iterable[str]) -> None:
    bad = [c for c in cols if df[c].isna().all()]
    assert not bad, f"Columns entirely NA: {bad}"

def type_parse(df: pd.DataFrame, date_cols: Iterable[str]=(), float_cols: Iterable[str]=()) -> pd.DataFrame:
    for c in date_cols:
        df[c] = pd.to_datetime(df[c], errors="coerce", utc=True)
    for c in float_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def quick_validation_report(df: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame({
        "dtype": df.dtypes.astype(str),
        "non_null": df.notna().sum(),
        "nulls": df.isna().sum(),
        "null_pct": (df.isna().mean()*100).round(2),
    })



## 1) API Pull — FRED (10Y Treasury `DGS10`)
**Endpoint:** `https://api.stlouisfed.org/fred/series/observations`  
**Auth:** `FRED_API_KEY` in `.env`  
**Params:** `series_id=DGS10`, `observation_start=2010-01-01`, `file_type=json`


In [ ]:

FRED_KEY = os.getenv("FRED_API_KEY")
assert FRED_KEY, "Put FRED_API_KEY in your .env (and never commit .env)."

def fred_series(series_id: str, start: str="2010-01-01", end: str|None=None) -> pd.DataFrame:
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        "series_id": series_id,
        "api_key": FRED_KEY,
        "file_type": "json",
        "observation_start": start,
    }
    if end:
        params["observation_end"] = end
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    j = r.json()
    obs = pd.DataFrame(j.get("observations", []))
    if obs.empty:
        raise ValueError(f"No observations returned for {series_id}")
    obs = obs.rename(columns={"date":"date", "value":"value"})[["date","value"]]
    obs = type_parse(obs, date_cols=["date"], float_cols=["value"])

    expect_columns(obs, ["date","value"])
    assert_no_all_na(obs, ["value"])
    assert len(obs) > 100, f"Unexpectedly few rows: {len(obs)}"
    return obs

api_df = fred_series("DGS10", start="2010-01-01")
display(api_df.head())
display(quick_validation_report(api_df))
api_path = save_csv(api_df, "api", "FRED", "DGS10")
api_path



## 2) Scrape — Wikipedia S&P 500 Table
**URL:** https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
We grab the first `wikitable` and validate presence of key columns.


In [ ]:

WIKI_URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

def scrape_sp500_table(url: str=WIKI_URL) -> pd.DataFrame:
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(r.text, "html.parser")
    table = soup.find("table", {"class": "wikitable"})
    assert table, "Could not find target table"
    headers = [th.get_text(strip=True) for th in table.find_all("tr")[0].find_all("th")]
    rows = []
    for tr in table.find_all("tr")[1:]:
        tds = [td.get_text(strip=True) for td in tr.find_all(["td","th"])]
        if len(tds) == len(headers):
            rows.append(tds)
    df = pd.DataFrame(rows, columns=headers)

    expected_subset = {"Symbol","Security","GICS Sector"}
    if expected_subset.isdisjoint(set(df.columns)):
        raise AssertionError(f"Expected any of {sorted(expected_subset)} in columns, got: {list(df.columns)[:6]}...")
    for c in ["Symbol","Security"]:
        if c in df.columns:
            assert_no_all_na(df, [c])
    return df

scrape_df = scrape_sp500_table()
display(scrape_df.head())
display(quick_validation_report(scrape_df).head(8))
scrape_path = save_csv(scrape_df, "scrape", "wikipedia", "sp500_table")
scrape_path



## 3) Documentation — Sources, Params, Validation, Assumptions
**API:** FRED observations endpoint for `DGS10` (10Y Treasury); params: `series_id`, `observation_start`, `file_type=json`; auth via `.env`.
**Scrape:** Wikipedia “List of S&P 500 companies” — first `wikitable`.
**Validation:** Column presence, dtype parsing (date/float), NA audit, and minimum row count for API.
**Assumptions/Risks:** Wikipedia structure may change; FRED returns missing values as '.' which are coerced to NaN.
